In [ ]:
!pip install nibabel nilearn torch torchvision einops tqdm matplotlib seaborn plotly pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 858.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!pip install numpy

In [ ]:
!pip install monai
!pip install torchsde  # For stochastic differential equations in diffusion models

# For downloading and handling the dataset
!pip install requests wget

# Print versions for documentation
import torch
import nibabel as nib
import nilearn
import numpy as np
import monai

print(f"PyTorch version: {torch.__version__}")
print(f"Nibabel version: {nib.__version__}")
print(f"Nilearn version: {nilearn.__version__}")
print(f"MONAI version: {monai.__version__}")

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"Memory cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

# Set seeds for reproducibility
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Create directories for data and results
import os
os.makedirs('data', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)

PyTorch version: 2.6.0+cu124
Nibabel version: 5.3.2
Nilearn version: 0.11.1
MONAI version: 1.4.0
Using device: cpu


In [ ]:
def check_versions():
    """Comprehensive version check for scientific Python packages"""
    import sys
    import importlib
    import os

    print("="*50)
    print("SYSTEM INFORMATION")
    print("="*50)
    print(f"Python version: {sys.version}")
    print(f"Python executable: {sys.executable}")
    print(f"Platform: {sys.platform}")

    # Check for multiple numpy installations
    print("\n" + "="*50)
    print("NUMPY INSTALLATION PATHS")
    print("="*50)
    try:
        !pip show numpy | grep "Location\|Version"
    except Exception as e:
        print(f"Error checking numpy installation: {e}")

    # Core Scientific Packages
    packages = [
        # Core scientific
        "numpy", "scipy", "pandas", "matplotlib", "scikit-learn",
        # Neuroimaging
        "nilearn", "nibabel",
        # Deep learning
        "torch", "monai"
    ]

    print("\n" + "="*50)
    print("PACKAGE VERSIONS")
    print("="*50)

    for package_name in packages:
        try:
            # Try to import the package
            package = importlib.import_module(package_name)
            version = getattr(package, "__version__", "unknown")
            print(f"{package_name}: {version}")

            # Special checks for numpy
            if package_name == "numpy":
                # Display numpy config information
                print("  NumPy include dirs:")
                try:
                    print(f"  {package.get_include()}")
                except Exception as e:
                    print(f"  Error getting NumPy include path: {e}")

                # Check for _ilp64 attribute
                try:
                    # First check if _umath_linalg exists
                    print("  NumPy linalg attributes:")
                    print(f"  Has _umath_linalg: {hasattr(package.linalg, '_umath_linalg')}")

                    if hasattr(package.linalg, '_umath_linalg'):
                        # Then check for _ilp64
                        has_ilp64 = hasattr(package.linalg._umath_linalg, '_ilp64')
                        print(f"  Has _ilp64 attribute: {has_ilp64}")
                except Exception as e:
                    print(f"  Error checking linalg attributes: {e}")
        except ImportError:
            print(f"{package_name}: Not installed")
        except Exception as e:
            print(f"{package_name}: Error importing - {str(e)}")

    # Test importing specific problematic modules
    print("\n" + "="*50)
    print("SPECIFIC MODULE IMPORT TESTS")
    print("="*50)

    # Test nilearn.plotting specifically
    try:
        import nilearn.plotting
        print("import nilearn.plotting: SUCCESS")
    except Exception as e:
        print(f"import nilearn.plotting: FAILED - {str(e)}")

    # Test nilearn.image specifically
    try:
        import nilearn.image
        print("import nilearn.image: SUCCESS")
    except Exception as e:
        print(f"import nilearn.image: FAILED - {str(e)}")

    # Test numpy.testing specifically
    try:
        import numpy.testing
        print("import numpy.testing: SUCCESS")
    except Exception as e:
        print(f"import numpy.testing: FAILED - {str(e)}")

    # Check for nilearn dependencies
    print("\n" + "="*50)
    print("NILEARN DEPENDENCIES")
    print("="*50)
    !pip show nilearn | grep "Requires"

# Run the version check
check_versions()

# Show pip installed packages (separately to avoid clutter)
print("\n" + "="*50)
print("PIP LIST (FILTERED FOR RELEVANT PACKAGES)")
print("="*50)
!pip list | grep -E "numpy|scipy|scikit|nilearn|nibabel|monai|torch"

SYSTEM INFORMATION
Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Python executable: /usr/bin/python3
Platform: linux

NUMPY INSTALLATION PATHS
Version: 1.26.4
Version 3.1, 31 March 2009
                       Version 3, 29 June 2007
  5. Conveying Modified Source Versions.
  14. Revised Versions of this License.
Location: /usr/local/lib/python3.11/dist-packages

PACKAGE VERSIONS
numpy: 1.23.5
  NumPy include dirs:
  /usr/local/lib/python3.11/dist-packages/numpy/core/include
  NumPy linalg attributes:
  Has _umath_linalg: True
  Has _ilp64 attribute: False
scipy: 1.14.1
pandas: 2.2.2
matplotlib: 3.10.0
scikit-learn: Not installed
nilearn: 0.11.1
nibabel: 5.3.2
torch: 2.6.0+cu124
monai: 1.4.0

SPECIFIC MODULE IMPORT TESTS
import nilearn.plotting: FAILED - module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'
import nilearn.image: SUCCESS
import numpy.testing: FAILED - module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

NILEARN DEPENDENCIES
Requires

In [ ]:
# Using DS000102 dataset from OpenNeuro - Visual categorization task
from nilearn.datasets import fetch_openneuro_dataset
import pandas as pd
import matplotlib.pyplot as plt
from nilearn import plotting, image

# Define dataset ID
dataset_id = 'ds000102'
data_dir = 'data/ds000102'

print("Downloading and loading dataset. This may take a few minutes...")

# Fetch the dataset
dataset = fetch_openneuro_dataset(dataset_id=dataset_id, data_dir=data_dir)

# Access paths for functional and anatomical data (first subject)
func_files = dataset.func
anat_files = dataset.anat

print(f"Number of functional files: {len(func_files)}")
print(f"Number of anatomical files: {len(anat_files)}")

# Sample the first functional file to explore
first_func_file = func_files[0]
print(f"First functional file: {first_func_file}")

# Load the file
func_img = nib.load(first_func_file)
func_data = func_img.get_fdata()

# Display basic info
print(f"Shape of functional data: {func_data.shape}")  # (X, Y, Z, time)
print(f"Data type: {func_data.dtype}")
print(f"Affine transformation:\n{func_img.affine}")
print(f"TR (Time Resolution): {func_img.header.get_zooms()[-1]} seconds")

# Plot some basic visualizations
plt.figure(figsize=(15, 5))

# 1. Plot middle slice of first timepoint
plt.subplot(131)
middle_z = func_data.shape[2] // 2
plt.imshow(func_data[:, :, middle_z, 0].T, cmap='viridis', origin='lower')
plt.colorbar()
plt.title('Middle slice (first timepoint)')

# 2. Plot time series for a voxel
plt.subplot(132)
middle_x, middle_y = func_data.shape[0] // 2, func_data.shape[1] // 2
plt.plot(func_data[middle_x, middle_y, middle_z, :])
plt.title(f'Time series for voxel ({middle_x}, {middle_y}, {middle_z})')
plt.xlabel('Timepoint')
plt.ylabel('BOLD signal')

# 3. Plot a simple brain mask
plt.subplot(133)
# Calculate a simple brain mask (voxels with values > 10% of max)
first_volume = func_data[:, :, :, 0]
simple_mask = first_volume > (0.1 * first_volume.max())
plt.imshow(simple_mask[:, :, middle_z].T, cmap='gray', origin='lower')
plt.title('Simple brain mask (middle slice)')

plt.tight_layout()
plt.show()

# Use nilearn's plotting for better visualization
print("\nBrain visualization using Nilearn:")
plotting.plot_epi(func_img, cmap='viridis', title='First volume of fMRI data')
plt.show()

# Show mean BOLD signal
mean_img = image.mean_img(func_img)
plotting.plot_stat_map(
    mean_img,
    bg_img=None,
    threshold=3,
    title='Mean BOLD signal',
    display_mode='mosaic'
)
plt.show()

# Access task information if available
try:
    events_file = [f for f in dataset.event if f.endswith('.tsv')][0]
    events_df = pd.read_csv(events_file, sep='\t')
    print("\nTask events sample:")
    print(events_df.head())

    # Plot event distribution
    plt.figure(figsize=(10, 6))
    events_df['trial_type'].value_counts().plot(kind='bar')
    plt.title('Distribution of trial types')
    plt.xlabel('Trial type')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Could not load event files: {e}")

AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'